In [1]:
from IPython.display import Image
Image(url='https://vesg.ipsl.upmc.fr/thredds/fileServer/IPSLFS/jservon/CliMAF_Notebooks_html/CliMAF-logo-small.png') 

### A science-oriented framework to ease the analysis of climate model simulations
##### WP5 ANR Convergence
##### Development team: Stéphane Sénési (CNRM-GAME), Gaëlle Rigoudy (CNRM-GAME), Jérôme Servonnat (LSCE-IPSL), Ludivine Vignon (CNRM-GAME), Laurent Franchisteguy (CNRM-GAME), Patrick Brockmann (LSCE-IPSL)
##### Beta-testing: Olivier Marti (LSCE-IPSL), Marie-Pierre Moine (CERFACS), Emilia Sanchez-Gomez (CERFACS)
##   
##### contact: climaf@meteo.fr
##### users list: climaf-users@meteo.fr

# Working with CliMAF on CORDEX ensembles
In this notebook we will see how to make a multi-model ensemble with CORDEX.
Compared with CMIP5 and CMIP6, CORDEX datasets are defined not only by one model but by a GCM/RCM couple (driving_model/model).

1. We will see how to make the most of CliMAF to work on those CORDEX ensembles.
2. and we will see how to access the land/sea mask

#  

## First, import climaf

In [2]:
from climaf.api import *

CliMAF version = 1.2.13


CliMAF install => /ciclad-home/jservon/Evaluation/CliMAF/climaf_installs/climaf_V1.2.13_post
python => /modfs/modtools/miniconda2/envs/analyse_2.7/bin/python
---
Required softwares to run CliMAF => you are using the following versions/installations:
ncl 6.3.0 => /opt/ncl-6.3.0/bin/ncl
cdo 1.9.6 => /opt/nco/1.9/bin/cdo
nco (ncks) 4.5.2 => /opt/nco-4.5.2/bin/ncks
ncdump fichier => /modfs/modtools/miniconda2/envs/analyse_2.7/bin/ncdump
Check stamping requirements
nco (ncatted) found -> /opt/nco-4.5.2/bin/ncatted
convert found -> /usr/bin/convert
pdftk found -> /usr/bin/pdftk
exiv2 found -> /ciclad-home/jservon/Evaluation/CliMAF/climaf_installs/climaf_V1.2.13_post/bin/exiv2
---


Cache directory set to : /data/jservon/climafcache_test (use $CLIMAF_CACHE if set) 
Cache directory for remote data set to : /data/jservon/climafcache_test/remote_data (use $CLIMAF_REMOTE_CACHE if set) 
Available macros read from ~/.climaf.macros are : []


## And set verbosity ('critical' -> minimum ; 'debug' -> maximum)

In [3]:
clog('critical') # min verbosity = critical < warning < info < debug = max verbosity

### ... and dont' forget to open the documentation in case you have questions.

### http://climaf.readthedocs.org/

### -> Use the "Quick search" space to search for what you are interested in, it is really powerfull!
#  
#  

## 1. Make a CORDEX ensemble
The specificity of CORDEX is that it works as GCM/RCM couples.
We propose here a way to build a multi-model CORDEX ensemble

In [4]:
from natsort import natsorted

# -- Let's make a dictionary to specify the general request:
req_dict  = dict(project = 'CORDEX',
                 CORDEX_domain = 'EUR-11',
                 frequency = 'daily',
                 experiment = 'historical',
                 period = '2000',
                 variable = 'tas'
               )

# -- Do the initial request to retrieve all the possible results:
req = ds(model = '*',
         driving_model = '*',
         realization = '*',
         **req_dict)

# -- We will now:
#      - make a loop on all the available 'driving_model' (GCMs)
#      - for each driving_model, we check the model (RCMs) available
#      - and for each GCM/RCM couple, we check the available realizations
#      - store all the matching datasets in myens_dict
#      - and create the ensemble with cens 
myens_dict = dict()
for GCM in req.explore('choices')['driving_model']:
    dumreq = ds(model='*',
                realization = '*',
                driving_model = GCM,
                **req_dict)
    RCMs = dumreq.explore('choices')['model']
    if not isinstance(RCMs, list):
        RCMs = [RCMs]
    for RCM in RCMs:
        dum2req = ds(model=RCM,
                     driving_model = GCM,
                     realization = '*',
                     **req_dict)
        print 'GCM = ',GCM,' RCM = ',RCM
        res_choices = dum2req.explore('choices')
        if res_choices:
            # -- If multiple realizations are available, we take the first one
            if isinstance(res_choices['realization'], list):
                dum2req.kvp['realization'] = natsorted(dum2req.explore('choices')['realization'])[0]
            myens_dict[GCM+'_'+RCM] = dum2req.explore('resolve')
#
my_CORDEX_ens = cens(myens_dict)

GCM =  CNRM-CERFACS-CNRM-CM5  RCM =  CLMcom-CCLM4-8-17
GCM =  CNRM-CERFACS-CNRM-CM5  RCM =  CNRM-ALADIN53
GCM =  CNRM-CERFACS-CNRM-CM5  RCM =  CNRM-ALADIN63
GCM =  CNRM-CERFACS-CNRM-CM5  RCM =  DMI-HIRHAM5
GCM =  CNRM-CERFACS-CNRM-CM5  RCM =  IPSL-WRF381P
GCM =  CNRM-CERFACS-CNRM-CM5  RCM =  KNMI-RACMO22E
GCM =  CNRM-CERFACS-CNRM-CM5  RCM =  RMIB-UGent-ALARO-0
GCM =  CNRM-CERFACS-CNRM-CM5  RCM =  SMHI-RCA4
GCM =  ICHEC-EC-EARTH  RCM =  CLMcom-CCLM4-8-17
GCM =  ICHEC-EC-EARTH  RCM =  CLMcom-ETH-COSMO-crCLIM-v1-1
GCM =  ICHEC-EC-EARTH  RCM =  DMI-HIRHAM5
GCM =  ICHEC-EC-EARTH  RCM =  IPSL-WRF381P
GCM =  ICHEC-EC-EARTH  RCM =  KNMI-RACMO22E
GCM =  ICHEC-EC-EARTH  RCM =  MOHC-HadREM3-GA7-05
GCM =  ICHEC-EC-EARTH  RCM =  SMHI-RCA4
GCM =  IPSL-IPSL-CM5A-LR  RCM =  GERICS-REMO2015
GCM =  IPSL-IPSL-CM5A-MR  RCM =  IPSL-WRF381P
GCM =  IPSL-IPSL-CM5A-MR  RCM =  KNMI-RACMO22E
GCM =  IPSL-IPSL-CM5A-MR  RCM =  SMHI-RCA4
GCM =  MOHC-HadGEM2-ES  RCM =  CLMcom-CCLM4-8-17
GCM =  MOHC-HadGEM2-ES  RCM = 

In [5]:
clog('critical')

In [6]:
summary(my_CORDEX_ens)

Keys - values:
{'domain': 'global', 'experiment': 'historical', 'institute': 'SMHI', 'version': 'latest', 'period': 2000, 'simulation': '', 'project': 'CORDEX', 'model_version': 'v1', 'frequency': 'daily', 'driving_model': 'NCC-NorESM1-M', 'realization': 'r1i1p1', 'variable': 'tas', 'model': 'SMHI-RCA4', 'CORDEX_domain': 'EUR-11', 'root': '/bdd'}
-- Ensemble members:
CNRM-CERFACS-CNRM-CM5_CLMcom-CCLM4-8-17
/bdd/CORDEX/output/EUR-11/CLMcom/CNRM-CERFACS-CNRM-CM5/historical/r1i1p1/CLMcom-CCLM4-8-17/v1/day/tas/latest/tas_EUR-11_CNRM-CERFACS-CNRM-CM5_historical_r1i1p1_CLMcom-CCLM4-8-17_v1_day_19960101-20001231.nc
--
CNRM-CERFACS-CNRM-CM5_CNRM-ALADIN53
/bdd/CORDEX/output/EUR-11/CNRM/CNRM-CERFACS-CNRM-CM5/historical/r1i1p1/CNRM-ALADIN53/v1/day/tas/latest/tas_EUR-11_CNRM-CERFACS-CNRM-CM5_historical_r1i1p1_CNRM-ALADIN53_v1_day_19960101-20001231.nc
--
CNRM-CERFACS-CNRM-CM5_CNRM-ALADIN63
/bdd/CORDEX/output/EUR-11/CNRM/CNRM-CERFACS-CNRM-CM5/historical/r1i1p1/CNRM-ALADIN63/v2/day/tas/latest/tas_EUR

/bdd/CORDEX/output/EUR-11/KNMI/MPI-M-MPI-ESM-LR/historical/r1i1p1/KNMI-RACMO22E/v1/day/tas/latest/tas_EUR-11_MPI-M-MPI-ESM-LR_historical_r1i1p1_KNMI-RACMO22E_v1_day_19960101-20001231.nc
--
MPI-M-MPI-ESM-LR_MOHC-HadREM3-GA7-05
/bdd/CORDEX/output/EUR-11/MOHC/MPI-M-MPI-ESM-LR/historical/r1i1p1/MOHC-HadREM3-GA7-05/v1/day/tas/latest/tas_EUR-11_MPI-M-MPI-ESM-LR_historical_r1i1p1_MOHC-HadREM3-GA7-05_v1_day_19960101-20001231.nc
--
MPI-M-MPI-ESM-LR_MPI-CSC-REMO2009
/bdd/CORDEX/output/EUR-11/MPI-CSC/MPI-M-MPI-ESM-LR/historical/r1i1p1/MPI-CSC-REMO2009/v1/day/tas/latest/tas_EUR-11_MPI-M-MPI-ESM-LR_historical_r1i1p1_MPI-CSC-REMO2009_v1_day_19960101-20001231.nc
--
MPI-M-MPI-ESM-LR_SMHI-RCA4
/bdd/CORDEX/output/EUR-11/SMHI/MPI-M-MPI-ESM-LR/historical/r1i1p1/SMHI-RCA4/v1a/day/tas/latest/tas_EUR-11_MPI-M-MPI-ESM-LR_historical_r1i1p1_SMHI-RCA4_v1a_day_19960101-20001231.nc
--
NCC-NorESM1-M_CLMcom-ETH-COSMO-crCLIM-v1-1
/bdd/CORDEX/output/EUR-11/CLMcom-ETH/NCC-NorESM1-M/historical/r1i1p1/CLMcom-ETH-COSMO-cr

## 2. Access the mask and land/area fraction sftlf: r0i0p0 and fx

In [7]:
from climaf.api import *

In [8]:
req_dict  = dict(project = 'CORDEX',
                 CORDEX_domain = 'EUR-11',
                 frequency = 'fx',
                 experiment = '*',
                 period = 'fx',
                 realization = 'r0i0p0',
                 variable = 'sftlf'
               )

# -- Do the initial request to retrieve all the possible results:
req = ds(model = '*',
         driving_model = '*',
         **req_dict)

req.explore('choices')

{'driving_model': ['CNRM-CERFACS-CNRM-CM5',
  'ECMWF-ERAINT',
  'ICHEC-EC-EARTH',
  'IPSL-IPSL-CM5A-LR',
  'IPSL-IPSL-CM5A-MR',
  'MIROC-MIROC5',
  'MOHC-HadGEM2-ES',
  'MPI-M-MPI-ESM-LR',
  'NCC-NorESM1-M',
  'NOAA-GFDL-GFDL-ESM2G'],
 'experiment': ['evaluation', 'historical', 'rcp26', 'rcp45', 'rcp85'],
 'institute': ['CLMcom', 'CLMcom-ETH', 'GERICS', 'MPI-CSC', 'SMHI', 'UHOH'],
 'model': ['CLMcom-CCLM4-8-17',
  'CLMcom-ETH-COSMO-crCLIM-v1-1',
  'GERICS-REMO2015',
  'MPI-CSC-REMO2009',
  'SMHI-RCA4',
  'UHOH-WRF361H'],
 'model_version': ['v1', 'v1a']}

### And of course, you can apply all the pretreatments you want to the members of your ensemble before getting your list of files.
For the available operators, we invite you to go deeper in CliMAF documentation and other CliMAF notebooks.


#   
## This was how to work with CliMAF ensembles!
##  
##  